# Preparación de conjuntos de datos para consultas

### Se preparan los csv necesarios para hacer las consultas en la API.

In [4]:
import pandas as pd
import numpy as np
import ast

## --> PlayTimeGenre
### ** DataFrames y Columnas  **


- A continuación se lista lo necesario para el primer endpoint :

    1. def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
        
        - df_games.Genre = (str)
        - df_games.Release_year = (int)
        - df_games.Id_game = (int)
        - df_items.Playtime_forever = (str)
             
        A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:
        - genre (str) = genero unico dentro de la lista total de generos
        - playtime_hours (int) = cantidad de horas de juego totales
        - year (int) = año de lanzamiento del juego 


In [5]:
# Se cargan los DataFrame necesarios
df_games = pd.read_csv("./data/games.csv")
df_items = pd.read_csv("./data/items.csv")


In [6]:
df_games.head(2)

,Id_game,App_name,Release_year,Genres,Specs,Price,Early_access,Developer
0,10,Counter-Strike,2000,['Action'],"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,0,Valve
1,20,Team Fortress Classic,1999,['Action'],"['Multi-player', 'Valve Anti-Cheat enabled']",4.99,0,Valve


In [7]:
df_items.head(2)

,Id_game,Item_name,Playtime_forever,Steam_id,User_id
0,10,Counter-Strike,6,76561197970982479,76561197970982479
1,10,Counter-Strike,904,76561198021501439,cuckmagnet


In [8]:
# Se crean los dataframes a utilizar para luego hacer el dataframe final para la consulta
df_1 = df_games[['Id_game','Genres','Release_year']].copy()
df_2 = df_items[['Id_game','Playtime_forever']].copy()

# Se realiza un inner join 
df_genre =  df_1.merge(df_2, on='Id_game', how='inner')
df_genre.head(2)

,Id_game,Genres,Release_year,Playtime_forever
0,10,['Action'],2000,6
1,10,['Action'],2000,904


In [9]:
# Se elimina la columna 'Id_game' ya que no es necesaria
df_genre = df_genre.drop(columns='Id_game')
df_genre.columns

Index(['Genres', 'Release_year', 'Playtime_forever'], dtype='object')

In [10]:
# Se cambian los valores 'No_Data' por 0
df_genre['Release_year'] = df_genre['Release_year'].replace('No_Data', 0).astype('int16')

In [11]:
# se eliminan los valores con 0
df_genre = df_genre[df_genre['Release_year'] != 0]

In [12]:
# Se eliminar registros en la columna 'playtime_forever' donde el valor es igual a 0
df_genre = df_genre[df_genre["Playtime_forever"] != 0]
df_genre = df_genre.reset_index(drop=True)

In [13]:
df_genre.head(2)

,Genres,Release_year,Playtime_forever
0,['Action'],2000,6
1,['Action'],2000,904


In [14]:
# Convertir las cadenas tipo lista en listas reales
df_genre['Genres'] = df_genre['Genres'].apply(ast.literal_eval)

In [15]:
# se desglosa la columna 'Genre' en filas separadas 
df_genre = df_genre.explode('Genres')

In [16]:
# Se suma el conteo de 'playtime_forever' agrupando por 'Genre' y 'Release_year' 
df_final_genre = df_genre.groupby(['Genres', 'Release_year'])['Playtime_forever'].sum().reset_index()

In [17]:
# Se pasa de minutos a horas y se cambia el tipo de dato de 'Playtime_forever' para que pese menos
df_final_genre['Hours']=(df_final_genre['Playtime_forever']/60).round().astype('int32')

In [18]:
# Se elimina la columna 'Playtime_forever' ya que no es necesaria
df_final_genre = df_final_genre.drop(columns='Playtime_forever')


In [19]:
df_final_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Genres        320 non-null    object
 1   Release_year  320 non-null    int16 
 2   Hours         320 non-null    int32 
dtypes: int16(1), int32(1), object(1)
memory usage: 4.5+ KB


In [20]:
# Se convierte a csv el DataFrame
df_final_genre.to_csv('end1.csv',index=False)

## --> UserForGenre
### ** DataFrames y Columnas  **


- A continuación se lista lo necesario para el segundo endpoint :

    2. def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

        
        - df_games.Genre = (str)
        - df_games.Id_game = (int)
        - df_items.Id_game = (int)
        - df_items.Playtime_forever = (str)
        - df_items.User_id = (str)
             
        A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:
        - user_id (str) = Id del usuario
        - playtime_hours (int) = cantidad de horas de juego totales
        - year (int) = año 

    ---> Con respecto a este endpoint dejo la inquietud dado a que lo se pide es muy ambiguo puesto que no se describe en ninguna parte el año en el que el usuario jugo cierta cantidad de horas de un determinado juego , por ende como la columna 'Playtime_forever' hace referencia al total de minutos jugados por un usuario, no los descrimina por año se sugiere cambien este endpoint por uno mas medible y real. para suplir el 'año' se tomara la fecha de lanzamiento de cada juego para asi tener un año de inicio, para coger los minutos totales y hacer el calculo de horas por año, se tiene como base que un jugador profesional entrena 6 a 8 horas diaras de lunes a sabado lo que significa al año juega un total de 1560 y 2496 horas, pero para fines mas practicos y no extender la tabla se define un tope de 2678 


### Playtime_forever por usuario

In [21]:
# Se cargan los DataFrame necesarios
df_games = pd.read_csv("./data/games.csv")
df_items = pd.read_csv("./data/items.csv")


In [22]:
# Secrea  una copia del df items y se asigna el otro nombre
time_user_game = df_items[['Id_game','User_id','Playtime_forever']].copy()
time_user_game.head(1)

,Id_game,User_id,Playtime_forever
0,10,76561197970982479,6


In [23]:
# se eliminan vfilas con valores 0 en playtime
time_user_game = time_user_game[time_user_game["Playtime_forever"] != 0]
time_user_game = time_user_game.reset_index(drop=True)

In [24]:
time_user_game.head(1)

,Id_game,User_id,Playtime_forever
0,10,76561197970982479,6


In [25]:
# se hace copia del df games con otro nombre
genres_id_year = df_games[['Id_game','Genres','Release_year']].copy()
genres_id_year.sample(10)

,Id_game,Genres,Release_year
9993,395910,['Adventure'],2010
19179,657770,"['Action', 'Casual', 'Indie']",2017
14284,509220,"['Casual', 'Indie']",2016
900,36320,"['Adventure', 'Casual']",2010
19100,655910,"['Action', 'Casual', 'Indie']",2017
18320,626170,"['Action', 'Adventure', 'Indie']",2017
9309,380360,"['Action', 'Casual', 'Free to Play', 'Indie', ...",2015
4740,279720,['RPG'],2004
2691,218090,"['Indie', 'Strategy']",2012
1867,204282,['Strategy'],2012


In [26]:
genres_id_year['Release_year'].unique()

array(['2000', '1999', '2003', '2001', '1998', '2004', '2010', '2006',
       '2007', '2008', '2009', '2011', '2012', '2005', '2002', '1994',
       '1995', '1996', '1997', '2013', '1992', '1993', '1990', 'No_Data',
       '2014', '2017', '1989', '2016', '2015', '1983', '1984', '1991',
       '1988', '2018', '1987', '1985', '1986', '2019', '2021'],
      dtype=object)

In [27]:
# Se verifica las filas que tienen 'No_Data'
No_Data = (genres_id_year['Release_year'] == 'No_Data').sum()
No_Data

33

In [28]:
# Se eliminan las filas 'No_Data'
genres_id_year=genres_id_year[genres_id_year['Release_year'] != 'No_Data']

In [29]:
# cambio de tipo de la variable Releaseyear
genres_id_year['Release_year'] = genres_id_year['Release_year'].astype('int16')

In [30]:
# Convertir las cadenas tipo lista en listas reales y en filas separadas 
genres_id_year['Genres'] = genres_id_year['Genres'].apply(ast.literal_eval)
genres_id_year = genres_id_year.explode('Genres')


In [31]:
# se unen los dos df en uno solo 
df_userMerge =  genres_id_year.merge(time_user_game, on='Id_game', how='inner')
df_userMerge.head(2)

,Id_game,Genres,Release_year,User_id,Playtime_forever
0,10,Action,2000,76561197970982479,6
1,10,Action,2000,cuckmagnet,904


In [32]:
# se realiza una agrupacion sumando el tiempo de cada jugador por categoria
df_final_UserGenre = df_userMerge.groupby(['Genres', 'Release_year','Id_game','User_id'])['Playtime_forever'].sum().reset_index()

In [33]:
df_final_UserGenre.head(1)

,Genres,Release_year,Id_game,User_id,Playtime_forever
0,Action,1983,227380,2Ta4,18


In [34]:
# Se pasan de minutos a horas 
df_final_UserGenre['Hours']=(df_final_UserGenre['Playtime_forever']/60).round().astype('int32')

In [35]:
# Se eliminan filas en donde horas sea = 0
df_final_UserGenre = df_final_UserGenre[df_final_UserGenre['Hours'] != 0]

In [36]:
# Se explora el valor que tiene mas horas
df_final_UserGenre['Hours'].max()

10713

In [37]:
# Se crea una copia de la columna hours para pruebas 
df_final_UserGenre['Original_Hours'] = df_final_UserGenre['Hours']

In [38]:
# Define una función personalizada para calcular la cantidad de horas por año ya que existen jugadores con muchas horas de juego
# se definio el tope como 2678 por año

def calcular_year2(row):
    if row['Hours'] > 2678: # si el valor es mayor a 278
        return row['Hours'] - 2678 # se asigana el valor de la resta entre el valor de la fila - 2678
    else:
        return 0  # Si no es asi coloca un 0 

# se ejecuta la funccion para la columna years que se creara
df_final_UserGenre['year2'] = df_final_UserGenre.apply(calcular_year2, axis=1) 

In [39]:
# Se establece el tope de horas por año a los usuarios que tengas mas de 2678 horas
df_final_UserGenre['Hours'] = df_final_UserGenre['Hours'].apply(lambda x: 2678 if x > 2678 else x)

In [40]:
# Define una función personalizada para calcular 'year3' haciendo el mismo proceso anterior

def calcular_year3(row):
    if row['year2'] > 2678:
        return row['year2'] - 2678
    else:
        return 0  # Si no cumple la condición, ponemos None

df_final_UserGenre['year3'] = df_final_UserGenre.apply(calcular_year3, axis=1)

In [41]:
df_final_UserGenre['year2'] = df_final_UserGenre['year2'].apply(lambda x: 2678 if x > 2678 else x)

In [42]:
# Define una función personalizada para calcular 'year4'

def calcular_year4(row):
    if row['year3'] > 2678:
        return row['year3'] - 2678
    else:
        return 0  # Si no cumple la condición, ponemos None

df_final_UserGenre['year4'] = df_final_UserGenre.apply(calcular_year4, axis=1)

In [43]:
df_final_UserGenre['year3'] = df_final_UserGenre['year3'].apply(lambda x: 2678 if x > 2678 else x)

In [44]:
# se cambia el nombre de la columna inial de hora para tener una similitud en las columnas
df_final_UserGenre = df_final_UserGenre.rename(columns={'Hours': 'year1'})

In [45]:
# Se cambian los tipos de variables de todas las columnas 
df_final_UserGenre['year1'] = df_final_UserGenre['year1'].astype('int16')
df_final_UserGenre['year2'] = df_final_UserGenre['year2'].astype('int16')
df_final_UserGenre['year3'] = df_final_UserGenre['year3'].astype('int16')
df_final_UserGenre['year4'] = df_final_UserGenre['year4'].astype('int16')

In [46]:
df_final_UserGenre.head(2)

,Genres,Release_year,Id_game,User_id,Playtime_forever,year1,Original_Hours,year2,year3,year4
1,Action,1983,227380,76561197966936422,331,6,6,0,0,0
3,Action,1983,227380,76561197969020980,98,2,2,0,0,0


In [47]:
# Se eliminan las columnas innecesarias 
df_final_UserGenre.drop(columns=['Id_game','Playtime_forever'], inplace=True)

In [48]:
# se ordenan las columnas a una manera mas legible
orden=['Genres','Release_year','User_id','Original_Hours','year1','year2','year3','year4',]
df_final_UserGenre = df_final_UserGenre[orden]

In [49]:
df_final_UserGenre

,Genres,Release_year,User_id,Original_Hours,year1,year2,year3,year4
1,Action,1983,76561197966936422,6,6,0,0,0
3,Action,1983,76561197969020980,2,2,0,0,0
4,Action,1983,76561197971401137,1,1,0,0,0
6,Action,1983,76561197975369524,1,1,0,0,0
9,Action,1983,76561197995164445,1,1,0,0,0
...,...,...,...,...,...,...,...,...
6568521,Web Publishing,2017,Eosoforcus,1,1,0,0,0
6568522,Web Publishing,2017,N47H4NI3L,27,27,0,0,0
6568523,Web Publishing,2017,dirklah,13,13,0,0,0
6568524,Web Publishing,2017,kushziller,4,4,0,0,0


Debido a que es muy grande la data  se empieza a depurar para que no consuma muchos recursos en la API

In [50]:
# Se depuran las filas en donde los dos primeros años tengan 0
df_final_UserGenre = df_final_UserGenre[(df_final_UserGenre['year1'] != 0) & (df_final_UserGenre['year2'] != 0)]

In [51]:
df_final_UserGenre

,Genres,Release_year,User_id,Original_Hours,year1,year2,year3,year4
25981,Action,2000,76561197967445425,3094,2678,416,0,0
26206,Action,2000,76561197985856321,3044,2678,366,0,0
26318,Action,2000,76561197996330176,4007,2678,1329,0,0
26412,Action,2000,76561198003688205,3421,2678,743,0,0
29780,Action,2000,andrewjohn,4036,2678,1358,0,0
...,...,...,...,...,...,...,...,...
6505853,Strategy,2015,76561198079518488,3082,2678,404,0,0
6506958,Strategy,2015,Ahahahahahahahahahahahahahahaha,3320,2678,642,0,0
6509455,Strategy,2015,idonothack,5558,2678,2678,202,0
6509579,Strategy,2015,jwatt1991,2987,2678,309,0,0


In [52]:
# Se convierte a csv el DataFrame
df_final_UserGenre.to_csv('end2.csv',index=False)

## --> UsersRecommend y UsersNotRecommend
### ** DataFrames y Columnas  **


- A continuación se lista lo necesario para el tercer y cuarto endpoint :

    3.  def UsersRecommend( año : int ): evuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.
        
        - df_games.Id_game = (int)
        - df_games.Release_year = (int)
        - df_games.App_name = (str)
        - df_review.Id_game = (int)
        - df_review.Recommended = (bool)
        - df_review.Sentiment_analysis = (int)
             
        A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:
        - App_name (str) = nombre del juego
        - Release_Year (int) = Año de lanzamiento del juego
        - Recommend (int) = Recomentacion positva o negativa del juego 
        - Sentiment_analysis (int) = Analisis de sentimiento (separado en 3 columnas 'Positivo','Malo','Neutral')
        
        <br>
        <br>
    4.  def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado.

        - df_games.Id_game = (int)
        - df_games.Release_year = (int)
        - df_games.App_name = (str)
        - df_review.Id_game = (int)
        - df_review.Recommended = (bool)
        - df_review.Sentiment_analysis = (int)
             
        A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:
        - App_name (str) = nombre del juego
        - Release_Year (int) = Año de lanzamiento del juego
        - Recommend (int) = Recomentacion positva o negativa del juego 
        - Sentiment_analysis (int) = Analisis de sentimiento (separado en 3 columnas 'Positivo','Malo','Neutral')
        

        


In [53]:
# Se cargan los DataFrame necesarios
df_games = pd.read_csv("./data/games.csv")
df_review = pd.read_csv("./data/review_sentiment.csv")

In [54]:
# Se crean los dataframes a utilizar para luego hacer el dataframe final para la consulta
df_g = df_games[['Id_game','App_name','Release_year']].copy()
df_r = df_review[['Id_game','Recommend','Sentiment_analysis',]].copy()

# Se realiza un inner join 
df_gr =  df_g.merge(df_r, on='Id_game', how='inner')
df_gr.head(10)

,Id_game,App_name,Release_year,Recommend,Sentiment_analysis
0,10,Counter-Strike,2000,1,1
1,10,Counter-Strike,2000,1,1
2,10,Counter-Strike,2000,1,1
3,10,Counter-Strike,2000,1,2
4,10,Counter-Strike,2000,1,2
5,10,Counter-Strike,2000,1,1
6,10,Counter-Strike,2000,1,2
7,10,Counter-Strike,2000,1,2
8,10,Counter-Strike,2000,1,1
9,10,Counter-Strike,2000,1,1


In [55]:
# Se crea una copia para el modelo de ML

df_gr_copia = df_gr[['Id_game','App_name','Recommend','Sentiment_analysis']]
df_gr_copia

,Id_game,App_name,Recommend,Sentiment_analysis
0,10,Counter-Strike,1,1
1,10,Counter-Strike,1,1
2,10,Counter-Strike,1,1
3,10,Counter-Strike,1,2
4,10,Counter-Strike,1,2
...,...,...,...,...
39518,423880,Carpe Diem,1,1
39519,423880,Carpe Diem,1,0
39520,423880,Carpe Diem,1,0
39521,423880,Carpe Diem,1,0


In [56]:
# Se elimina la columna innecesario 'Id_game'
df_gr.drop(columns=['Id_game'], inplace=True)

In [57]:
# Se aplica un one-hot encoding a la columna 'Sentiment_analysis'
dummies_review = pd.get_dummies(df_gr['Sentiment_analysis'], prefix='Sentiment_analysis')

In [58]:
# Se concatena el df original con el df antteriormente creado 'dummies...'
df_gr = pd.concat([df_gr, dummies_review], axis=1)
df_gr


,App_name,Release_year,Recommend,Sentiment_analysis,Sentiment_analysis_0,Sentiment_analysis_1,Sentiment_analysis_2
0,Counter-Strike,2000,1,1,False,True,False
1,Counter-Strike,2000,1,1,False,True,False
2,Counter-Strike,2000,1,1,False,True,False
3,Counter-Strike,2000,1,2,False,False,True
4,Counter-Strike,2000,1,2,False,False,True
...,...,...,...,...,...,...,...
39518,Carpe Diem,2015,1,1,False,True,False
39519,Carpe Diem,2015,1,0,True,False,False
39520,Carpe Diem,2015,1,0,True,False,False
39521,Carpe Diem,2015,1,0,True,False,False


In [59]:
# Se elimina la columna 'Sentiment_analysis' y se renombrar unas columnas
df_gr = df_gr.drop('Sentiment_analysis', axis=1)
df_gr = df_gr.rename(columns={'Sentiment_analysis_0': 'Malo','Sentiment_analysis_1': 'Neutral','Sentiment_analysis_2': 'Positivo' })

In [60]:
# Se cambia el typo de dato a las columnas anteriormente nombradas para reducir el peso de df
df_gr['Malo'] = df_gr['Malo'].astype('int8')
df_gr['Neutral'] = df_gr['Neutral'].astype('int8')
df_gr['Positivo'] = df_gr['Positivo'].astype('int8')

In [61]:
df_gr.sample(20)

,App_name,Release_year,Recommend,Malo,Neutral,Positivo
21519,LEGO® Batman 2 DC Super Heroes™,2012,1,0,0,1
36150,Farming Simulator 15,2014,1,0,1,0
10658,Just Cause 2,2010,1,0,1,0
33850,Amazing World™,2014,1,0,0,1
16231,Terraria,2011,1,0,1,0
10030,Garry's Mod,2006,1,0,0,1
27116,Dungeon Defenders II,2017,1,0,0,1
459,Counter-Strike: Source,2004,1,0,0,1
14030,Stronghold Kingdoms,2012,0,0,0,1
34234,Robocraft,2017,1,1,0,0


In [62]:
# Se convierte a csv el DataFrame
df_gr.to_csv('end3.csv',index=False)

## --> sentiment_analysis
### ** DataFrames y Columnas  **


- A continuación se lista lo necesario para el quinto endpoint :

    5. def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas categorizados con un análisis de sentimiento.
        
        - df_games.Release_year = (int)
        - df_games.Id_game = (int)
        - df_review.Id_game = (int)
        - df_review.Sentiment_analysis = (int)
             
        A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:

        - Release_Year (int) = año de lanzamiento del juego
        - Sentiment_analysis (int) = Analisis de sentimiento (separado en 3 columnas 'Positivo','Malo','Neutral')

In [63]:
# Se usa el DataFrame anteriormente creado y se usaran las columnas necesarias 
df_sent = df_gr[['Release_year','Malo','Neutral','Positivo']]

In [64]:
# Se agrupa el DataFrame por año y se suma los valores de cada columna por año 
df_sent_agru = df_sent = df_sent.groupby('Release_year', as_index=False).sum()
df_sent_agru.head(2)

,Release_year,Malo,Neutral,Positivo
0,1989,0,1,0
1,1990,0,1,4


In [65]:
# Se convierte a csv el DataFrame
df_sent_agru.to_csv('end5.csv',index=False)

## --> Modelo ML

Se creara un DataFrame con las columnas necesarias para el modelo de ML 

In [66]:
# Se crea el DF
ml = df_gr_copia
ml.head(3)

,Id_game,App_name,Recommend,Sentiment_analysis
0,10,Counter-Strike,1,1
1,10,Counter-Strike,1,1
2,10,Counter-Strike,1,1


Con base a las columnas 'Recommend,Malo,Neutral y Positivo' se buscara crear una puntuacion para cada juego teniendo en cuenta la combinacion de estas columnas y se le asiganra una escala de la siguiente manera:

- **1** si en la columna Malo tiene (1) y Recommend tiene (1 o 0)
- **2** si en la columna Neutral tiene (1) y Recommend tiene (0)
- **3** si en la columna Neutral tiene (1) y Recommend (1)
- **4** si en la columna Positivo tiene (1) y Recommend (0)
- **5** si en la columna Positivo tiene (1) y Recommend (1)

Esto se hace para poder filtrar los mejores juegos

In [67]:
def escala(row):

    if row["Sentiment_analysis"] == 0 and not row["Recommend"]:
        return 1
    elif row["Sentiment_analysis"] == 0 and row["Recommend"]:
        return 1
    elif row["Sentiment_analysis"] == 1 and not row["Recommend"]:
        return 2
    elif row["Sentiment_analysis"] == 1 and row["Recommend"]:
        return 3
    elif row["Sentiment_analysis"] == 2 and not row["Recommend"]:
        return 4
    elif row["Sentiment_analysis"] == 2 and row["Recommend"]:
        return 5
    else:
        return None

In [68]:
ml['Score'] = ml.apply(escala,axis =1)
ml.sample(1)

,Id_game,App_name,Recommend,Sentiment_analysis,Score
21678,214490,Alien: Isolation,1,1,3


In [69]:
# Se filtra el DF en donde solo se dejan los juegos con score 5
top_score = ml[ml['Score']>4]
top_score

,Id_game,App_name,Recommend,Sentiment_analysis,Score
3,10,Counter-Strike,1,2,5
4,10,Counter-Strike,1,2,5
6,10,Counter-Strike,1,2,5
7,10,Counter-Strike,1,2,5
17,10,Counter-Strike,1,2,5
...,...,...,...,...,...
39500,418340,RWBY: Grimm Eclipse,1,2,5
39501,418910,Idle Civilization,1,2,5
39509,423880,Carpe Diem,1,2,5
39514,423880,Carpe Diem,1,2,5


In [70]:
# funcion para  agrupar por "Id_game" manteniendo el nombre del juego 
def first_app_name(series):
    return series.iloc[0]  # Obtenemos el primer valor de la serie

# se aplica la funcion al df grupo
top_score = top_score.groupby(["Id_game", "Score"])["App_name"].apply(first_app_name).reset_index()

In [71]:
top_score.sample(5)

,Id_game,Score,App_name
468,219190,5,Ultratron
290,67370,5,The Darkness II
177,25910,5,Supreme Ruler 2020 Gold
507,224500,5,Gnomoria
1059,341000,5,Mad Games Tycoon


In [72]:
# Se anexa la columna Genres del DF games 
top_score = top_score.merge(df_games[['Id_game','Genres']],on='Id_game', how='left')
top_score.sample(7)

,Id_game,Score,App_name,Genres
385,206210,5,Gotham City Impostors Free to Play,"['Action', 'Free to Play']"
491,221810,5,The Cave,['Adventure']
370,204180,5,Waveform,"['Adventure', 'Indie', 'Strategy']"
477,220240,5,Far Cry 3,"['Action', 'Adventure']"
665,247000,5,Talisman: Digital Edition,"['Indie', 'RPG', 'Strategy']"
229,41700,5,S.T.A.L.K.E.R.: Call of Pripyat,"['Action', 'RPG']"
577,235360,5,Prime World: Defenders,"['Casual', 'Indie', 'RPG', 'Strategy']"


In [73]:
# Se eliminan caracteres de la columna 'Genres'

def eliminar_caracteres(cadena):
    # Reemplaza los caracteres no deseados por una cadena vacía
    caracteres_a_eliminar = ["[", "]", "'", " ", '"']
    for caracter in caracteres_a_eliminar:
        cadena = cadena.replace(caracter, '')

    return cadena

top_score['Genres'] =top_score['Genres'].apply(eliminar_caracteres)

In [74]:
# se realiza una funcion para eliminar palabras repetidas 

def eliminar_palabras_repetidas(cadena):
    cadena = cadena.split(",")  # se divide la cadena por medio del seprador ","  quedando uan lista de palabras
    palabras_unicas = list(set(cadena))  # Se convierte la lista en conjunto y automaticamente elimina las palabras duplicadas
    texto_sin_repetidos = ','.join(palabras_unicas) # Se convierte el conjunto a cadena uniendo las palabras por "," y un espacio
    return texto_sin_repetidos

    # Se aplica 
top_score['Genres'] =top_score['Genres'].apply(eliminar_palabras_repetidas)

In [75]:
# Se pone en minuscula todo el contenido de la columna 'Genres'
top_score['Genres'] =top_score['Genres'].str.lower()

In [76]:
top_score.sample(5)

,Id_game,Score,App_name,Genres
91,10090,5,Call of Duty: World at War,action
313,95400,5,ibb & obb,"indie,action,adventure"
65,6910,5,Deus Ex: Game of the Year Edition,action
1116,367120,5,Seduce Me the Otome,"casual,simulation"
61,6060,5,"Star Wars: Battlefront 2 (Classic, 2005)",action


In [77]:
top_score

,Id_game,Score,App_name,Genres
0,10,5,Counter-Strike,action
1,20,5,Team Fortress Classic,action
2,30,5,Day of Defeat,action
3,40,5,Deathmatch Classic,action
4,50,5,Half-Life: Opposing Force,action
...,...,...,...,...
1183,418070,5,Turbo Pug,"indie,casual"
1184,418340,5,RWBY: Grimm Eclipse,"indie,action,adventure"
1185,418910,5,Idle Civilization,"simulation,strategy,earlyaccess,indie,rpg"
1186,423880,5,Carpe Diem,"indie,casual,freetoplay"


In [78]:
top_score.to_csv("datos_ML.csv")